# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [95]:
from bs4 import BeautifulSoup
import requests
html_page = requests.get('http://books.toscrape.com/') # Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser') # Pass the page contents to beautiful soup for parsing

In [3]:
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:29" name="created"/>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="static/oscar/favicon.ico" rel="shortcut icon"/>
  <link href="static/oscar/css/styles.css" rel="stylesheet" type="tex

In [17]:
soup.findAll('h3')[0].findAll('a')[0].attrs['title']
# Find the path to extracting each title before setting up list comprehensions

'A Light in the Attic'

In [26]:
def retrieve_titles(soup):
    grandparents = soup.findAll('h3')
    parents = [grandparent.findAll('a') for grandparent in grandparents]
    titles = [parent[0].attrs['title'] for parent in parents]
    return titles

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [31]:
# Finding the path to extract each rating
import re
regex = re.compile("star-rating (.*)")
soup.findAll('p', {"class" : regex})[0].attrs['class'][-1]

'Three'

In [34]:
def retrieve_ratings(soup):
    import re
    regex = re.compile("star-rating (.*)")
    str_ratings = [i.attrs['class'][-1] for i in soup.findAll('p', {"class" : regex})]
    str_to_int = {'One':1, 'Two':2, 'Three':3, 'Four':4, 'Five':5}
    ratings = [str_to_int[i] for i in str_ratings]
    return ratings

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [44]:
float(soup.findAll('p','price_color')[0].text[1:])

51.77

In [47]:
def retrieve_prices(soup):
    return [float(i.text[1:]) for i in soup.findAll('p','price_color')]

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [53]:
soup.findAll('p','instock availability')[0].text.strip()

'In stock'

In [54]:
def retrieve_availabilities(soup):
    return [i.text.strip() for i in soup.findAll('p','instock availability')]

In [61]:
import pandas as pd
column_data = [retrieve_titles(soup),retrieve_ratings(soup),retrieve_prices(soup),retrieve_availabilities(soup)]
df = pd.DataFrame(column_data).transpose()
df.columns = ['Title', 'Rating', 'Price (in Pounds)', 'Availability']
df.head()

,Title,Rating,Price (in Pounds),Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock


In [69]:
test_df = pd.DataFrame(columns=['Title', 'Rating', 'Price (in Pounds)', 'Availability'])
pd.concat([test_df,df.iloc[6:10,:]],axis=0)

,Title,Rating,Price (in Pounds),Availability
6,The Dirty Little Secrets of Getting Your Dream...,4,33.34,In stock
7,The Coming Woman: A Novel Based on the Life of...,3,17.93,In stock
8,The Boys in the Boat: Nine Americans and Their...,4,22.6,In stock
9,The Black Maria,1,52.15,In stock


## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [80]:
#books_df = pd.DataFrame(columns=['Title', 'Rating', 'Price (in Pounds)', 'Availability'])
html_page = requests.get(f'https://books.toscrape.com/catalogue/page-2.html')
soup = BeautifulSoup(html_page.content, 'html.parser')
column_data = [retrieve_titles(soup),retrieve_ratings(soup),retrieve_prices(soup),retrieve_availabilities(soup)]
df = pd.DataFrame(column_data).transpose()
df.columns = ['Title', 'Rating', 'Price (in Pounds)', 'Availability']
books_df = pd.concat([books_df,df],axis=0).reset_index(drop=True)
books_df

,Title,Rating,Price (in Pounds),Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
5,The Requiem Red,1,22.65,In stock
6,The Dirty Little Secrets of Getting Your Dream...,4,33.34,In stock
7,The Coming Woman: A Novel Based on the Life of...,3,17.93,In stock
8,The Boys in the Boat: Nine Americans and Their...,4,22.6,In stock
9,The Black Maria,1,52.15,In stock


In [87]:
books_df = pd.DataFrame(columns=['Title', 'Rating', 'Price (in Pounds)', 'Availability'])
for i in range(1,51):
    html_page = requests.get(f'https://books.toscrape.com/catalogue/page-{i}.html')
    soup = BeautifulSoup(html_page.content, 'html.parser')
    column_data = [retrieve_titles(soup),retrieve_ratings(soup),retrieve_prices(soup),retrieve_availabilities(soup)]
    df = pd.DataFrame(column_data).transpose()
    df.columns = ['Title', 'Rating', 'Price (in Pounds)', 'Availability']
    books_df = pd.concat([books_df,df],axis=0).reset_index(drop=True)

In [88]:
len(books_df)

1000

## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [140]:
books_df = pd.DataFrame(columns=['Title', 'Rating', 'Price (in Pounds)', 'Availability'])
url = 'https://books.toscrape.com/catalogue/page-1.html'

while True:
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    column_data = [retrieve_titles(soup),retrieve_ratings(soup),retrieve_prices(soup),retrieve_availabilities(soup)]
    df = pd.DataFrame(column_data).transpose()
    df.columns = ['Title', 'Rating', 'Price (in Pounds)', 'Availability']
    books_df = pd.concat([books_df,df],axis=0).reset_index(drop=True)
    if soup.findAll('ul','pager')[0].find('li','next') != None:
        page = soup.findAll('ul','pager')[0].find('li','next').find('a')['href']
        url = 'https://books.toscrape.com/catalogue/' + page
        continue
    else:
        break


In [141]:
len(books_df)

1000

## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!